In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, confusion_matrix, \
    precision_score, recall_score, f1_score
from scipy.stats import mode

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# Drop features from other locations
df.drop(['PM_Caotangsi', 'PM_Shahepu'], axis=1, inplace=True)

# Drop features 'precipitation' and 'Iprec'
df.drop(['No', 'precipitation', 'Iprec'], axis=1, inplace=True)

# Check for missing values
print(df.isnull().sum() / df.shape[0] * 100)

In [ ]:
# For the most crucial feature, 'PM_US Post', we check whether more than 85% of data is missing for any year.
# If more than 85% of data is missing, then that year may not be considered relevant.
gbdf = df.groupby(by='year').agg('count')
del_year = gbdf.index[gbdf['PM_US Post'] / gbdf['month'] < 0.15]
df = df[~df['year'].isin(del_year)]
print(df['year'].unique()) # 2010, 2011 are dropped


In [ ]:
df['PM_US Post'].fillna(method='ffill', inplace=True)
df.dropna(axis=0, inplace=True)
# Provera da li ima jos nedostajucih vrednosti
print(df.isna().sum())

In [ ]:
# Frequency distribution
plt.figure(figsize=(10, 6))
plt.hist(df['PM_US Post'], bins=30, color='red', edgecolor='black')
plt.title('Distribution of PM2.5 concentration')
plt.xlabel('PM2.5 concetration')
plt.ylabel('Number of samples')
plt.grid(True)
plt.show()

In [ ]:
# Central tendency measures
mean_value = np.mean(df['PM_US Post'])
median_value = np.median(df['PM_US Post'])
mode_value = mode(df['PM_US Post'], keepdims = True).mode[0]

print("Mean:", mean_value)
print("Median:", median_value)
print("Mode:", mode_value)

In [ ]:
# Calculating IQR
Q1 = df['PM_US Post'].quantile(0.25)
Q3 = df['PM_US Post'].quantile(0.75)
IQR = Q3 - Q1

# Define borders for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 5 * IQR

# Identfication of outliers
outliers = df[(df['PM_US Post'] < lower_bound) | (df['PM_US Post'] > upper_bound)]['PM_US Post']

print("Identificated outliers:")
print(outliers)

In [ ]:
# Visual representation of outlier values
plt.figure(figsize=(10, 6))
plt.boxplot(df['PM_US Post'], vert=False, patch_artist=True)
plt.title('Boxplot for PM2.5 concentration')
plt.xlabel('Concentration PM2.5')
plt.yticks([1], ['PM_US Post'])
plt.grid(True)
plt.axvline(lower_bound, color='r', linestyle='dotted', linewidth=2, label=f'Outlier border (lower): {lower_bound:.2f}')
plt.axvline(upper_bound, color='r', linestyle='dotted', linewidth=2, label=f'Outlier border (upper): {upper_bound:.2f}')
plt.show()

In [ ]:
# Dependance between season and PM2.5 concentration
season_mapping = {1: 'Spring', 2: 'Summer', 3: 'Autumn', 4: 'Winter'}

plt.figure(figsize=(10, 6))
plt.plot(df['season'], df['PM_US Post'], marker='o', linestyle='-', color='b')
plt.title('Dependance between season and PM2.5 concentration')
plt.xlabel("Season")
plt.ylabel("PM2.5 concentration \n (µg/m3)")
plt.xticks(df['season'].unique(), [season_mapping[season] for season in df['season'].unique()])

plt.grid(True)
plt.show()

In [ ]:
# Dependance between year and PM2.5 concentration
plt.figure(figsize=(10, 6))
plt.plot(df['year'], df['PM_US Post'], marker='o', linestyle='-', color='b')
plt.title('Dependance between year and PM2.5 concentration')
plt.xlabel('Year')
plt.ylabel('PM2.5 concentration \n (µg/m3)')
plt.grid(True)
plt.show()

In [ ]:
# Dependance between DEWP and PM2.5 concentration
plt.figure(figsize=(10, 6))
plt.plot(df['DEWP'], df['PM_US Post'], marker='o', linestyle='-', color='b')
plt.title('Dependance between DEWP and PM2.5 concentration')
plt.xlabel("DEWP")
plt.ylabel("PM2.5 concentration \n (µg/m3)")
plt.grid(True)
plt.show()

In [ ]:
# Dependance between HUMI and PM2.5 concentration
plt.figure(figsize=(10, 6))
plt.plot(df['HUMI'], df['PM_US Post'], marker='o', color='b')
plt.title('Dependance between HUMI and PM2.5 concentration')
plt.xlabel("HUMI")
plt.ylabel("PM2.5 concentration \n (µg/m3)")
plt.grid(True)
plt.show()

In [ ]:
# Dependance between TEMP and PM2.5 concentration
plt.figure(figsize=(10, 6))
plt.scatter(df['TEMP'], df['PM_US Post'], marker='o', linestyle='-', color='b')
plt.title('Dependance between TEMP and PM2.5 concentration')
plt.xlabel("cbwd")
plt.ylabel("PM2.5 concentration \n (µg/m3)")
plt.grid(True)
plt.show()

In [ ]:
# Dependance between Iws and PM2.5 concentration
plt.figure(figsize=(10, 6))
plt.plot(df['Iws'], df['PM_US Post'], marker='o', linestyle='-', color='b')
plt.title('Dependance between Iws and PM2.5 concentration')
plt.xlabel("Iws")
plt.ylabel("PM2.5 concentration \n (µg/m3)")
plt.grid(True)
plt.show()


In [ ]:
# Correlation matrix
corr_mat = df.corr(numeric_only=True)
sb.heatmap(corr_mat, annot=True)
plt.show()

In [ ]:
df.describe()

In [ ]:
# Convert categorical features into numerical
df.loc[df['cbwd'] == 'cv', 'cbwd'] = 0
df.loc[df['cbwd'] == 'SW', 'cbwd'] = 1
df.loc[df['cbwd'] == 'SE', 'cbwd'] = 2
df.loc[df['cbwd'] == 'NW', 'cbwd'] = 3
df.loc[df['cbwd'] == 'NE', 'cbwd'] = 4

In [ ]:
x = df.drop(['PM_US Post'], axis=1).copy()
y = df['PM_US Post'].copy()

# Create test and training set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, train_size=0.7, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [ ]:
def model_evaluation(y, y_predicted, N, d):
    mse = mean_squared_error(y_test, y_predicted)
    mae = mean_absolute_error(y_test, y_predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_predicted)
    r2_adj = 1 - (1 - r2) * (N - 1) / (N - d - 1)

    print('Mean squared error: ', mse)
    print('Mean absolute error: ', mae)
    print('Root mean squared error: ', rmse)
    print('R2 score: ', r2)
    print('R2 adjusted score: ', r2_adj)

    res = pd.concat([pd.DataFrame(y.values), pd.DataFrame(y_predicted)], axis=1)
    res.columns = ['y', 'y_pred']
    print(res.head(20))

In [ ]:
# 1) Linear regression with hypothesis y=b0+b1x1+b2x2+...+bnxn
first_regression_model = LinearRegression(fit_intercept=True)
first_regression_model.fit(x_train, y_train)
y_predicted = first_regression_model.predict(x_test)
model_evaluation(y_test, y_predicted, x_train.shape[0], x_train.shape[1])

plt.figure(figsize=(10, 5))
plt.bar(range(len(first_regression_model.coef_)), first_regression_model.coef_)
plt.show()

print("Coefficients: ", first_regression_model.coef_)


In [ ]:
# Feature standardization - Z normalization
scaler = StandardScaler()
scaler.fit(x_train)

x_train_std = scaler.transform(x_train)
x_test_std = scaler.transform(x_test)

x_train_std = pd.DataFrame(x_train_std)
x_test_std = pd.DataFrame(x_test_std)

x_train_std.columns = list(x.columns)
x_test_std.columns = list(x.columns)

In [ ]:
# Linear regression with hypothesis y=b0+b1x1+b2x2+...+bnxn+c1x1x2+c2x1x3+...+d1x1^2+d2x2^2+...+dnxn^2
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
x_inter_train = poly.fit_transform(x_train_std)
x_inter_test = poly.transform(x_test_std)

regression_model_degree = LinearRegression()
regression_model_degree.fit(x_inter_train, y_train)
y_predicted = regression_model_degree.predict(x_inter_test)

model_evaluation(y_test, y_predicted, x_inter_train.shape[0], x_inter_train.shape[1])

plt.figure(figsize=(10, 5))
plt.bar(range(len(regression_model_degree.coef_)), regression_model_degree.coef_)
plt.show()
print("Coefficients", regression_model_degree.coef_)

In [ ]:
# Ridge regression
ridge_model = Ridge(alpha=4)
ridge_model.fit(x_inter_train, y_train)
y_predicted = ridge_model.predict(x_inter_test)

model_evaluation(y_test, y_predicted, x_inter_train.shape[0], x_inter_train.shape[1])

plt.figure(figsize=(10, 5))
plt.bar(range(len(ridge_model.coef_)), ridge_model.coef_)
plt.show()
print("Coefficients - Ridge: ", ridge_model.coef_)

# Prikazivanje koeficijenata
sb.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sb.scatterplot(x=y_test, y=y_predicted, color='blue', alpha=0.7, label='Predicted values')
sb.scatterplot(x=y_test, y=y_test, color='red', alpha=0.7, label='Real values')
plt.xlabel("Real values")
plt.ylabel("Predicted values")
plt.title("Real vs Predicted values")
plt.show()

In [ ]:
# Adding a column where we will describe the danger based on PM2.5 particle levels.
new_df = df.assign(danger='NAN')
# Adding values to the 'danger' column based on the values of the 'PMUS Post' column.
new_df.loc[new_df['PM_US Post'] < 55.5, 'danger'] = 'safe'
new_df.loc[(new_df['PM_US Post'] > 55.4) & (new_df['PM_US Post'] < 150.5), 'danger'] = 'unsafe'
new_df.loc[new_df['PM_US Post'] > 150.4, 'danger'] = 'dangerous'

# Converting categorical features into numerical features.
new_df.loc[new_df['danger'] == 'safe', 'danger'] = 0
new_df.loc[new_df['danger'] == 'unsafe', 'danger'] = 1
new_df.loc[new_df['danger'] == 'dangerous', 'danger'] = 2

new_df.loc[new_df['cbwd'] == 'cv', 'cbwd'] = 0
new_df.loc[new_df['cbwd'] == 'SW', 'cbwd'] = 1
new_df.loc[new_df['cbwd'] == 'SE', 'cbwd'] = 2
new_df.loc[new_df['cbwd'] == 'NW', 'cbwd'] = 3
new_df.loc[new_df['cbwd'] == 'NE', 'cbwd'] = 4

Xtr = new_df.iloc[:, :-1] 
Ytr = new_df.iloc[:, -1].astype('int')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Xtr, Ytr, test_size=0.30, train_size=0.70, random_state=42)

In [ ]:
# Confusion matrix for the 'safe' class
def evaluation_classif_class_safe(conf_mat):
    TPc = conf_mat[0, 0]
    TNc1 = conf_mat[1, 1]
    TNc2 = conf_mat[1, 2]
    TNc3 = conf_mat[2, 1]
    TNc4 = conf_mat[2, 2]
    FPc1 = conf_mat[1, 0]
    FPc2 = conf_mat[2, 0]
    FNc1 = conf_mat[0, 1]
    FNc2 = conf_mat[0, 2]

    precision = TPc / (TPc + FPc1 + FPc2)
    accuracy = (TPc + TNc1 + TNc2 + TNc3 + TNc4) / (TPc + TNc1 + TNc2 + TNc3 + TNc4 + FPc1 + FPc2 + FNc1 + FNc2)
    sensitivity = TPc / (TPc + FPc1 + FPc2)
    specificity = (TNc1 + TNc2 + TNc3 + TNc4) / (TNc1 + TNc2 + TNc3 + TNc4 + FPc1 + FPc2)
    F_score = 2 * precision * sensitivity / (precision + sensitivity)
    print("Class safe: ")
    print('precision: ', precision)
    print('accuracy: ', accuracy)
    print('sensitivity/recall: ', sensitivity)
    print('specificity: ', specificity)
    print('F score: ', F_score)
    print("")
    return accuracy


# Confusion matrix for the 'unsafe' class
def evaluation_classif_class_unsafe(conf_mat):
    TPp = conf_mat[1, 1]
    TNp1 = conf_mat[0, 0]
    TNp2 = conf_mat[0, 2]
    TNp3 = conf_mat[2, 0]
    TNp4 = conf_mat[2, 2]
    FPp1 = conf_mat[0, 1]
    FPp2 = conf_mat[2, 1]
    FNp1 = conf_mat[1, 0]
    FNp2 = conf_mat[1, 2]

    precision1 = TPp / (TPp + FPp1 + FPp2)
    accuracy1 = (TPp + TNp1 + TNp2 + TNp3 + TNp4) / (TPp + TNp1 + TNp2 + TNp3 + TNp4 + FPp1 + FPp2 + FNp1 + FNp2)
    sensitivity1 = TPp / (TPp + FPp1 + FPp2)
    specificity1 = (TNp1 + TNp2 + TNp3 + TNp4) / (TNp1 + TNp2 + TNp3 + TNp4 + FPp1 + FPp2)
    F_score1 = 2 * precision1 * sensitivity1 / (precision1 + sensitivity1)
    print("Class unsafe: ")
    print('precision: ', precision1)
    print('accuracy: ', accuracy1)
    print('sensitivity/recall: ', sensitivity1)
    print('specificity: ', specificity1)
    print('F score: ', F_score1)
    print("")
    return accuracy1


# Confusion matrix for the 'dangerous' class
def evaluation_classif_class_dangerous(conf_mat):
    TPps = conf_mat[2, 2]
    TNps1 = conf_mat[0, 0]
    TNps2 = conf_mat[0, 1]
    TNps3 = conf_mat[1, 0]
    TNps4 = conf_mat[1, 1]
    FPps1 = conf_mat[0, 2]
    FPps2 = conf_mat[1, 2]
    FNps1 = conf_mat[2, 0]
    FNps2 = conf_mat[2, 1]
    precision2 = TPps / (TPps + FPps1 + FPps2)
    accuracy2 = (TPps + TNps1 + TNps2 + TNps3 + TNps4) / (
            TPps + TNps1 + TNps2 + TNps3 + TNps4 + FPps1 + FPps2 + FNps1 + FNps2)
    sensitivity2 = TPps / (TPps + FPps1 + FPps2)
    specificity2 = (TNps1 + TNps2 + TNps3 + TNps4) / (TNps1 + TNps2 + TNps3 + TNps4 + FPps1 + FPps2)
    F_score2 = 2 * precision2 * sensitivity2 / (precision2 + sensitivity2)
    print("Class dangerous: ")
    print('precision: ', precision2)
    print('accuracy: ', accuracy2)
    print('sensitivity/recall: ', sensitivity2)
    print('specificity: ', specificity2)
    print('F score: ', F_score2)
    print("")
    return accuracy2


In [ ]:
# kNN classifier on training set
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  
acc = []
best_accuracy = 0
best_params = {'metric': '', 'weights': '', 'n_neighbors': 0}

for m in ['euclidean', 'manhattan', 'chebyshev']:
    for d in ['distance', 'uniform']:
        for i in range(1, 14):
            indexes = kf.split(x_train, y_train)
            acc_t = []
            fin_conf_mat = np.zeros((len(np.unique(y_train)), len(np.unique(y_train))))
            for train_index, test_index in indexes:
                knn = KNeighborsClassifier(n_neighbors=i, metric=m, weights=d)
                knn.fit(x_train.iloc[train_index, :], y_train.iloc[train_index])
                y_pred = knn.predict(x_train.iloc[test_index, :])
                acc_t.append(accuracy_score(y_train.iloc[test_index], y_pred))
                fin_conf_mat += confusion_matrix(y_train.iloc[test_index], y_pred, labels=[0, 1, 2])
            print('metric =', m, ', weight =', d, 'neighbor - i =', i, 'precision', np.mean(acc_t),
                  ' \na mat. konf. je:\n', fin_conf_mat)

            avg_accuracy = ((evaluation_classif_class_safe(fin_conf_mat) + evaluation_classif_class_unsafe(
                fin_conf_mat) + evaluation_classif_class_dangerous(fin_conf_mat)) / 3)
            print("Average accuracy of classifier: \n", avg_accuracy)
            acc.append(np.mean(acc_t))
            if avg_accuracy > best_accuracy:
                best_accuracy = avg_accuracy
                best_params = {'metric': m, 'weights': d, 'n_neighbors': i}
                
print('Best accuracy:', best_accuracy)
print('Best parameters:', best_params)            

In [ ]:
# kNN classifier on test set
classifierknn = KNeighborsClassifier(n_neighbors=12, metric='manhattan', weights='distance')
classifierknn.fit(x_train, y_train)
y_pred1 = classifierknn.predict(x_test)
conf_mat1 = confusion_matrix(y_test, y_pred1)

print('Final confusion matrix: ')
print(conf_mat1)

avg_accuracy = ((evaluation_classif_class_safe(conf_mat1) + evaluation_classif_class_unsafe(
    conf_mat1) + evaluation_classif_class_dangerous(conf_mat1)) / 3)

print("Average accuracy: ", avg_accuracy)
print('Micro precision: ', precision_score(y_test, y_pred1, average='micro'))
print('Macro precision: ', precision_score(y_test, y_pred1, average='macro'))
print('Micro sensitivity/recall: ', recall_score(y_test, y_pred1, average='micro'))
print('Macro sensitivity/recall: ', recall_score(y_test, y_pred1, average='macro'))
print('Micro F score: ', f1_score(y_test, y_pred1, average='micro'))
print('Macro F score: ', f1_score(y_test, y_pred1, average='macro'))